In [38]:
from bs4 import BeautifulSoup
import random,string
import pandas as pd

soup=BeautifulSoup(open('cfc_19100108_102.htm'),'html.parser')
tables = soup.find_all('table')
all_dfs = []

for table in tables:

    headers=table.find_all('th')
    n_col = 0
    n_row_tot = 0
    
    # find number of rows and columns to create a dataframe
    for row in table.find_all('tr'):
        cols = row.find_all(['td'])
        if len(cols)>0:
            n_row_tot+=1
            if len(cols) > n_col:
                n_col = len(cols)
    n_row=0
    
    # find number of rows that the column headers will occupy
    for row in table.find_all('tr'):
        if len(row.find_all(['th'])) != 0:
            n_row = n_row + 1

    
    skip_index = [0 for i in range(0, n_col)]
    total_arr = []
    i=0
    
    # logic to read the column headers and frame the multi index object for pandas df
    for m in range(0,n_row):
        arr = [None for i in range(0,n_col)]
        arr_c=0
        while arr_c < n_col:
            rs = headers[i].get("rowspan")
            cs = headers[i].get("colspan")

            if rs is None and cs is None:
                if skip_index[arr_c] != 0:
                    skip_index[arr_c]=skip_index[arr_c]-1
                    arr[arr_c]=''
                    arr_c = arr_c + 1
                else:
                    arr[arr_c]=headers[i].get_text().strip()
                    arr_c = arr_c + 1
                    if i != len(headers)-1:
                        i = i+1
                    else:
                        break

            if rs is not None:
                if skip_index[arr_c] == 0:
                    skip_index[arr_c] = int(rs)-1
                    arr[arr_c]=headers[i].get_text().strip()
                    arr_c = arr_c + 1
                    if i != len(headers)-1:
                        i = i+1
                    else:
                        break

            if cs is not None:
                for a in range(arr_c,arr_c+int(cs)):
                    arr[a] = headers[i].get_text().strip()
                    arr_c=sum(x is not None for x in arr)
                if i != len(headers)-1:
                        i = i+1
                else:
                    break
            arr_c=sum(x is not None for x in arr)
        total_arr.append(arr)

    new_table = pd.DataFrame(index = range(0, n_row_tot),columns=pd.MultiIndex.from_arrays(total_arr))
    row=0
    
    # logic to fill data in the df (assumption is that data is available per row per column)
    for each_row in table.find_all('tr'):
        data = each_row.find_all('td')
        if len(data) != 0:
            col=0
            for cell in data:
                new_table.iloc[row,col]=cell.get_text().strip()
                col=col+1
            row = row+1 
    all_dfs.append(new_table)

final_dfs=[]

# logic to find out which of the tables are identical and merge them into one
while len(all_dfs) != 0:
    same = []
    i=1
    m=0
    if len(all_dfs) != 1:
        for i in range(1,len(all_dfs)):
            if all_dfs[0].columns.equals(all_dfs[i].columns):
                m=m+1
                same.append(all_dfs[0])
                same.append(all_dfs[i])
    if m==0:
        final_dfs.append(all_dfs[0])   
    del all_dfs[0]
    for a in all_dfs:
        if len(same) != 0:
            if same[0].columns.equals(a.columns):
                all_dfs.remove(a)

    if len(same)!=0:
        joined_df = pd.concat(same)
        final_dfs.append(joined_df)

 
writer=pd.ExcelWriter('output.xlsx')
for tab in final_dfs:
    tab.to_excel(writer,''.join(random.choice(string.ascii_lowercase) for i in range(5)))
writer.save()